**Input files and variables**

*   **Get meteorological model (coordenates and variables)**
*   **Select metar and oberserved variable**

*   **Variable observed: intervals and labels**

**Output**
 
*   **Show meteorological model nearests points map**
*   **Observed variable climatology**







In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

#get coordenates
coor=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LEST_st/database/distan_lat42.898lon-8.418p5R4Km.csv")

#load model
model = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LEST_st/database/lat42.898lon-8.418p5R4KmD0.csv",
                    parse_dates=["time"]).set_index("time")

#load metar variable
variable_metar = "visibility_o"
var = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LEST_st/database/LEST.csv",
                  usecols=["time",variable_metar],parse_dates=["time"]).set_index("time")   

#match model and variable observed drop nan. 
all = pd.concat([model,var],axis=1).dropna()

#variable to string interval. and new column :variable observed labeled
interval=pd.IntervalIndex.from_tuples([(-0.1,1000), (1000, 40000)])
labels=['<= 1000 m', '> 1000 m']
all["var_o_l"]=pd.cut(all.iloc[:,-1], bins=interval,retbins=False,
                        labels=labels).map({a:b for a,b in zip(interval,labels)}).astype(str)
#model variables
X = all.iloc[:,:-2]

#observed variable
Y = all.iloc[:,-1:]

#show results observed variable
print("Observed variable results")
print(Y["var_o_l"].value_counts())
print("***************")
print(Y["var_o_l"].value_counts(normalize=True))

#show map
px.set_mapbox_access_token("pk.eyJ1IjoiZ3JhbmFudHVpbiIsImEiOiJja3B4dGU4OTkwMTFmMm9ycnNhMjJvaGJqIn0.VWzx_PkD9A5cSUVsn_ijCA")
px.scatter_mapbox(coor, hover_data=['distance'],lat='lat', lon='lon',color='distance', title="Nearest points",
                           color_continuous_scale=px.colors.cyclical.IceFire,)

Observed variable results
> 1000 m     64917
<= 1000 m     1650
Name: var_o_l, dtype: int64
***************
> 1000 m     0.975213
<= 1000 m    0.024787
Name: var_o_l, dtype: float64


**Compare model variable (several points) labeled and observed variable**

In [ ]:
from sklearn.metrics import classification_report
import math
from scipy.stats import entropy

#label meteorological model vis points 0 to n_points
var_model = "visibility"
n_points = 5


for p in range (0,n_points):
  X[var_model+str(p)+"_l"] = pd.cut(X[var_model+str(p)], bins=interval,retbins=False,
                                  labels=labels).map({a:b for a,b in zip(interval,labels)}).astype(str)
  print("Model point "+str(p)+" results\n",X[var_model+str(p)+"_l"].value_counts(normalize=True))

  table = pd.crosstab(Y.var_o_l, X[var_model+str(p)+"_l"],margins=True,)
  print("Confusion matrix\n",table)
  

  # Heidke Skill Score (HSS). Binary labels
  if len(labels) == 2:
    a = table.values[0,0]
    b = table.values[1,0]
    c = table.values[0,1]
    d = table.values[1,1]
    hss = 2*(a*d-b*c)/((a+c)*(c+d)+(a+b)*(b+d))
    print(" Heidke Skill Score:",round(hss,2))

  
  column_sc=pd.crosstab(Y.var_o_l,X[var_model+str(p)+"_l"] , margins=True,normalize="columns")
  column_sc=column_sc.append(pd.DataFrame(entropy(column_sc,base=2)/(math.log2(column_sc.shape[0])),columns=["entropy/entropy.max"],
                                          index=column_sc.columns).T).rename(columns={"All":"Climatology"})
  print ("Precision and entropy meteorologic model\n",column_sc) 
  print("Quality report meteorologic model\n",pd.DataFrame(classification_report(Y.var_o_l, X[var_model+str(p)+"_l"],output_dict=True)).T)                                         
  print("**************************************************************")

#delete columns model variable labeled
X = X.drop(list(X)[-n_points:], axis=1)  

Model point 0 results
 > 1000 m     0.908048
<= 1000 m    0.091952
Name: visibility0_l, dtype: float64
Confusion matrix
 visibility0_l  <= 1000 m  > 1000 m    All
var_o_l                                  
<= 1000 m            732       918   1650
> 1000 m            5389     59528  64917
All                 6121     60446  66567
 Heidke Skill Score: 0.16
Precision and entropy meteorologic model
 visibility0_l        <= 1000 m  > 1000 m  Climatology
<= 1000 m             0.119588  0.015187     0.024787
> 1000 m              0.880412  0.984813     0.975213
entropy/entropy.max   0.528176  0.113489     0.167534
Quality report meteorologic model
               precision    recall  f1-score       support
<= 1000 m      0.119588  0.443636  0.188393   1650.000000
> 1000 m       0.984813  0.916986  0.949690  64917.000000
accuracy       0.905253  0.905253  0.905253      0.905253
macro avg      0.552201  0.680311  0.569041  66567.000000
weighted avg   0.963367  0.905253  0.930820  66567.000000
**

**Machine learning**
1.   **Correlation observed variable and model variables**




In [ ]:
#Correlation target and features
pd.options.display.max_rows=999

corre = all.corr().loc[variable_metar].sort_values()
corre_f = pd.DataFrame(corre[~corre.between(-.35, .35, inclusive=False)])
print(corre_f)
print(corre_f.index)


              visibility_o
cfl2             -0.387654
cfl0             -0.387489
cfl1             -0.386692
cfl3             -0.384712
cfl4             -0.383594
rh4              -0.354194
rh3              -0.353213
rh2              -0.351426
visibility3       0.357524
visibility1       0.365982
visibility0       0.382084
visibility4       0.407318
visibility_o      1.000000
Index(['cfl2', 'cfl0', 'cfl1', 'cfl3', 'cfl4', 'rh4', 'rh3', 'rh2',
       'visibility3', 'visibility1', 'visibility0', 'visibility4',
       'visibility_o'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.



**Input**


*   **X variables from metereological model**
*   **Resample unbalanced variables SMOTE library**
*   **Scaled variables**
*   **PCA n components**


*   **AI model selection and tune**




**Output**


*   **Trained AI model**
*   **Trained AI model score**







In [ ]:
#Run the machine learning model
from scipy.stats import entropy
import math
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#PCA number
pca_n = 4

#with or without PCA
raw_data = True

#select model variables
"""
#x_var = ['visibility0', 'visibility1', 'visibility4', 'lwflx2', 'visibility9',
       'mod0', 'dir2', 'dir9', 'cin7', 'rh3', 'wind_gust9', 'shflx7', 'dir8',
       'cft3', 'cin8', 'lhflx8', 'cfl9', 'visibility3', 'temp3', 'dir4',
       'cfl4', 'lwflx4', 'dir6', 'visibility2']
"""       
x_var = X.columns

#split data test and train
X_train, X_test, y_train, y_test = train_test_split(X[x_var],Y.values.reshape(-1, 1),
                                                    test_size=0.1,)

# Resample
X_res, y_res =SMOTE().fit_resample(X_train,y_train)

#scale Xtrain
scale = StandardScaler().fit(X_res)
Xs_train = scale.transform(X_res)

#pca
pca = PCA(n_components=pca_n).fit(Xs_train)
pca_train= pca.transform(Xs_train)


#fit model
if raw_data:
  #ml_model = ExtraTreesClassifier(n_estimators=150).fit(X_res,y_res)
  ml_model = LGBMClassifier(n_estimators=250).fit(X_res,y_res)
  #ml_model = MLPClassifier(hidden_layer_sizes=(100,100,50), random_state=1, max_iter=300).fit(X_res,y_res)
  y_pred = ml_model.predict(X_test)
else:
  #ml_model = ExtraTreesClassifier(n_estimators=150).fit(pca_train,y_res)
  ml_model = MLPClassifier(hidden_layer_sizes=(100,100,50), random_state=1, max_iter=300).fit(pca_train,y_res)
  #ml_model=LGBMClassifier(n_estimators=250).fit(X_train,y_train)
  y_pred = ml_model.predict(pca.transform(scale.transform(X_test)))


table = pd.crosstab( y_test.reshape(1,-1)[0],y_pred,margins=True,)
print("Confusion matrix\n",table)
print("**************************************")

column_sc=pd.crosstab(y_test.reshape(1,-1)[0],y_pred , margins=True,normalize="columns")
column_sc=column_sc.append(pd.DataFrame(entropy(column_sc,base=2)/(math.log2(column_sc.shape[0])),columns=["entropy/entropy.max"],
                                        index=column_sc.columns).T).rename(columns={"All":"Climatology"})

print ("Precision and entropy AI\n",column_sc)  

# Heidke Skill Score (HSS). Binary labels
if len(labels) == 2:
  a = table.values[0,0]
  b = table.values[1,0]
  c = table.values[0,1]
  d = table.values[1,1]
  hss = 2*(a*d-b*c)/((a+c)*(c+d)+(a+b)*(b+d))
  print(" Heidke Skill Score:",round(hss,2))

print("**************************************")
print("Quality report AI\n",pd.DataFrame(classification_report(y_test.reshape(1,-1)[0],y_pred,output_dict=True)).T)



Confusion matrix
 col_0      <= 1000 m  > 1000 m   All
row_0                               
<= 1000 m         94        80   174
> 1000 m         154      6329  6483
All              248      6409  6657
**************************************
Precision and entropy AI
 col_0                <= 1000 m  > 1000 m  Climatology
<= 1000 m             0.379032  0.012482     0.026138
> 1000 m              0.620968  0.987518     0.973862
entropy/entropy.max   0.957356  0.096834     0.174637
 Heidke Skill Score: 0.43
**************************************
Quality report AI
               precision    recall  f1-score      support
<= 1000 m      0.379032  0.540230  0.445498   174.000000
> 1000 m       0.987518  0.976246  0.981849  6483.000000
accuracy       0.964849  0.964849  0.964849     0.964849
macro avg      0.683275  0.758238  0.713673  6657.000000
weighted avg   0.971613  0.964849  0.967830  6657.000000


**Cross validation**

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate

print("cross validation. waiting...")
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=100)
if raw_data:
  cros_val_dict=cross_validate(ml_model, X, Y.values.reshape(1, -1)[0], cv=cv,scoring=["accuracy",'f1_macro',"f1_weighted"]) 
else:
  cros_val_dict=cross_validate(ml_model, pca.fit_transform(scale.fit_transform(X)), Y.values.reshape(1, -1)[0], cv=cv,scoring=["accuracy",'f1_macro',"f1_weighted"])
cros_val=pd.DataFrame(cros_val_dict)  
print("f1_weighted: %0.2f (+/- %0.2f)" % (cros_val_dict['test_f1_weighted'].mean(), cros_val_dict['test_f1_weighted'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (cros_val_dict['test_accuracy'].mean(), cros_val_dict['test_accuracy'].std() * 2))

cross validation. waiting...
f1_weighted: 0.93 (+/- 0.00)
Accuracy: 0.95 (+/- 0.00)


**Export model**

In [ ]:
import pickle
model_dict={"x_var":x_var,"ml_model":ml_model,"coor":coor}
pickle.dump(model_dict, open("vis_LEST_d0.al", 'wb'))